<a href="https://colab.research.google.com/github/Geebigib/Data-Science-Project/blob/main/XAI_from_ECOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
!unzip /content/drive/MyDrive/SWaT_Dataset_v0.zip -d /content/

Archive:  /content/drive/MyDrive/SWaT_Dataset_v0.zip
  inflating: /content/SWaT_Dataset_Attack_v0.csv  
  inflating: /content/SWaT_Dataset_Normal_v0.csv  


In [4]:
swat_attack = pd.read_csv("SWaT_Dataset_Attack_v0.csv")
swat_attack.columns = swat_attack.columns.str.replace(' ','')
swat_attack.set_index('Timestamp',inplace = True)

In [5]:
feature = swat_attack.columns.values
feature = feature[feature!='Normal/Attack']
feature

array(['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202',
       'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204',
       'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302',
       'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401',
       'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503',
       'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601',
       'P601', 'P602', 'P603'], dtype=object)

In [6]:
path = '/content/drive/MyDrive/Data_sci_Project/XAI_result/'

In [7]:
with open(path+'ECOD_3.pkl', 'rb') as f:
    x = pickle.load(f)

In [8]:
true_anomaly = pd.read_csv("/content/drive/MyDrive/Data_sci_Project/List_of_attacks_Final.csv")

In [9]:
true_attack = true_anomaly[~true_anomaly["End Time"].isnull()]

In [10]:
true_attack.tail(3)

,Attack #,Start Time,End Time,Attack Point,Start State,Attack,Actual Change,Expected Impact or attacker intent,Unexpected Outcome
38,39,2/01/2015 11:43:48,11:50:28,"FIT-401, AIT-502",In Normal Range,Set value of FIT-401 as 0.5; Set value of AIT-...,No,UV will shut down and water will go to RO,UV did not shutdown
39,40,2/01/2015 11:51:42,11:56:38,FIT-401,In Normal Range,Set value of FIT-401 as 0,No,UV will shut down and water will go to RO,"P-402 did not close, both should be interlinked"
40,41,2/01/2015 13:13:02,13:40:56,LIT-301,Water level between L and H,decrease value by 0.5 mm per second,No,Tank overflow,Rate of decrease in water level reduced after ...


In [11]:
#att found from DeepSVDD
att_no = " 1  2  3  4  8 11 13 17 19 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41".split()

In [12]:
len(att_no)

30

In [13]:
att_point = np.where(true_attack["Attack #"].isin(att_no),true_attack["Attack Point"], None)

In [14]:
att_point = att_point[att_point != None]

In [15]:
att_point = [e.replace("-","") for e in att_point]

In [16]:
att_point = [e.split(", ") for e in att_point]

In [17]:
att_point

[['MV101'],
 ['P102'],
 ['LIT101'],
 ['MV504'],
 ['DPIT301'],
 ['FIT401'],
 ['MV304'],
 ['MV303'],
 ['AIT504'],
 ['MV101', 'LIT101'],
 ['UV401', 'AIT502', 'P501'],
 ['P602', 'DIT301', 'MV302'],
 ['P203', 'P205'],
 ['LIT401', 'P401'],
 ['P101', 'LIT301'],
 ['P302', 'LIT401'],
 ['P302'],
 ['P201', 'P203', 'P205'],
 ['LIT101', 'P101', 'MV201'],
 ['LIT401'],
 ['LIT301'],
 ['LIT101'],
 ['P101'],
 ['P101; P102'],
 ['LIT101'],
 ['P501', 'FIT502'],
 ['AIT402', 'AIT502'],
 ['FIT401', 'AIT502'],
 ['FIT401'],
 ['LIT301']]

##SHAP

In [18]:
shap = x['shap_values'].copy()

In [19]:
len(shap)

30

In [20]:
shap[0]

array([4.64398897, 2.17824493, 2.48959585, 1.51984693, 0.        ,
       2.80043685, 2.21349898, 2.21425075, 2.64987916, 4.11549064,
       0.        , 0.        , 2.80920846, 0.        , 3.62952393,
       0.        , 4.80477965, 1.53413882, 1.04601311, 2.233329  ,
       2.10001318, 2.55920089, 1.7256375 , 0.        , 1.60597928,
       3.36847794, 2.26364012, 0.63353001, 2.30048374, 0.        ,
       0.        , 0.        , 0.        , 0.        , 2.87527321,
       2.79300461, 1.68485574, 2.76141917, 2.46562218, 1.92045889,
       0.88126795, 2.03381182, 0.        , 0.        , 1.01719873,
       5.08356624, 0.84038992, 2.7501706 , 0.        , 1.31474998,
       0.        ])

In [21]:
shap_val = pd.DataFrame(index=feature)
for i in range(len(att_no)):
  shap_val[f"shap_att_{att_no[i]}"] = shap[i]

In [22]:
shap_val

,shap_att_1,shap_att_2,shap_att_3,shap_att_4,shap_att_8,shap_att_11,shap_att_13,shap_att_17,shap_att_19,shap_att_21,...,shap_att_32,shap_att_33,shap_att_34,shap_att_35,shap_att_36,shap_att_37,shap_att_38,shap_att_39,shap_att_40,shap_att_41
FIT101,4.643989,0.000000,3.145124,3.945960,4.544596,5.818275,2.275583,3.218720,4.134635,8.962476,...,8.839846,6.345512,9.706605,3.971470,17.694735,7.832939,4.211087,18.842053,14.323983,7.244147
LIT101,2.178245,0.000000,3.152299,1.155219,5.056017,2.447768,2.505499,3.334622,2.731108,1.010422,...,8.643729,7.701330,12.240292,7.265452,3.725774,16.079122,6.330581,29.043839,18.237064,6.561311
MV101,2.489596,3.321999,1.980016,4.279870,4.574072,4.678390,2.847177,3.403511,4.879739,4.690774,...,8.458289,7.851682,8.493332,9.936358,23.925429,13.016546,3.802853,18.336320,26.724494,10.724646
P101,1.519847,2.128003,1.952120,2.367788,1.094228,7.131443,2.178723,4.291445,8.136400,2.678453,...,7.903930,8.029017,9.511011,9.184544,15.701659,9.190340,3.924145,9.789086,16.653729,8.894883
P102,0.000000,2.184987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,13.423300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AIT201,2.800437,0.000000,3.948706,0.881549,1.800112,2.308422,1.901587,1.252745,3.831060,3.394346,...,5.395066,6.030110,8.307214,10.866757,11.796097,13.534148,3.562791,14.691674,22.418580,7.837035
AIT202,2.213499,1.620405,1.412435,2.116928,1.861652,3.290058,1.494560,1.427322,1.708587,4.350347,...,11.916857,4.327308,15.067207,12.664989,8.264138,13.875227,7.519915,9.527829,22.959281,12.320080
AIT203,2.214251,0.000000,5.898698,1.421302,5.552361,6.933171,3.065001,1.439215,12.380746,3.852277,...,10.125054,5.840377,13.643947,9.077614,19.780934,18.261365,4.115325,13.190447,11.617192,7.863404
FIT201,2.649879,0.000000,3.405414,2.929538,5.106094,7.132209,2.130771,3.999445,4.454057,3.294473,...,7.171817,10.567499,16.106218,12.191030,10.202374,15.266813,10.643915,8.913518,25.693439,3.780468
MV201,4.115491,0.032540,4.978681,2.198555,7.158364,4.943867,2.511051,3.621130,1.335445,5.887115,...,15.263231,6.340479,8.846440,8.982768,11.204508,18.778966,5.945996,12.771440,11.083702,10.371192


In [25]:
IOU5 = []
IOU10 = []
acc = []
for i in range(len(att_no)):
  positive = shap_val[shap_val["shap_att_"+str(att_no[i])] >0 ]["shap_att_"+str(att_no[i])].sort_values(ascending=False)
  top5 = positive[0:5]
  top10 = positive[0:10]
  IOU5.append(sum(top5.index.isin(att_point[i]))/5)
  IOU10.append(sum(top10.index.isin(att_point[i]))/10)
  acc.append(sum(positive.index.isin(att_point[i]))/len(att_point[i]))


In [27]:
print("IOU5: ",(sum(IOU5)/len(att_no))*100)
print("IOU10: ",(sum(IOU10)/len(att_no))*100)
print("ACC: ",(sum(acc)/len(att_no))*100)

IOU5:  6.666666666666667
IOU10:  2.0
ACC:  87.77777777777777


##ALE

In [72]:
ale = x['ales'].copy()

In [31]:
ale_val = pd.DataFrame(index=feature)
for i in range(len(att_no)):
  ale_val[f"ale_val{att_no[i]}"] = ale[i]

In [71]:
IOU5 = []
IOU10 = []
acc = []
for i in range(len(att_no)):
  positive = ale_val[ale_val["ale_val"+str(att_no[i])] >0 ]["ale_val"+str(att_no[i])].sort_values(ascending=False)
  top5 = positive[0:5]
  top10 = positive[0:10]
  IOU5.append(sum(top5.index.isin(att_point[i]))/5)
  IOU10.append(sum(top10.index.isin(att_point[i]))/10)
  acc.append(sum(positive.index.isin(att_point[i]))/len(att_point[i]))

print("IOU5: ",(sum(IOU5)/len(att_no))*100)
print("IOU10: ",(sum(IOU10)/len(att_no))*100)
print("ACC: ",(sum(acc)/len(att_no))*100)

IOU5:  3.457244023359688
IOU10:  5.257117833165037
ACC:  30.265915941990013


##LIME

In [73]:
lime = x['limes'].copy()

In [74]:
lime_re = []
lime_score = []
for i in range(len(lime)):
  a =[]
  score = []
  for e in  range(len(lime[i])):
    if lime[i][e][1] > 0:
      a.append(lime[i][e][0])
      score.append(lime[i][e][1])
  lime_re.append(a)
  lime_score.append(score)

In [75]:
lime_re[0]

['AIT401 > 1.00',
 'AIT402 <= 0.20',
 'LIT101 > 0.74',
 'FIT101 > 0.93',
 'PIT502 > 0.50',
 'FIT201 > 0.99',
 'PIT503 <= 0.35',
 'LIT301 <= 0.26',
 'AIT503 <= 0.40',
 'PIT501 <= 0.35',
 'AIT502 <= 0.16']

In [76]:
lime_score[0]

[1.641905303893302,
 1.3201119168922715,
 1.2670084694149801,
 1.2546373492072471,
 1.0985940960431395,
 1.0942010458172986,
 1.0518934403131082,
 0.9900146504995108,
 0.9853516352070676,
 0.9626430726755052,
 0.8418493440090861]

In [77]:
import re
features = []
for b in lime_re:
  att = []
  for i in b:
    a = re.split('[<= ]', i)
    for e in a:
      if re.search('[a-zA-Z]', e):
        att.append(e)
  features.append(att)
features[0]

['AIT401',
 'AIT402',
 'LIT101',
 'FIT101',
 'PIT502',
 'FIT201',
 'PIT503',
 'LIT301',
 'AIT503',
 'PIT501',
 'AIT502']

In [78]:
len(features)

30

In [79]:
IOU5 = []
IOU10 = []
acc = []
for i in range(len(att_no)):
  positive = pd.DataFrame(lime_score[i], index=features[i],columns=["lime"]).sort_values(by ="lime",ascending=False)
  top5 = positive[0:5]
  top10 = positive[0:10]
  IOU5.append(sum(top5.index.isin(att_point[i]))/5)
  IOU10.append(sum(top10.index.isin(att_point[i]))/len(top10))
  acc.append(sum(positive.index.isin(att_point[i])))


print("IOU5: ",(sum(IOU5)/len(att_no))*100)
print("IOU10: ",(sum(IOU10)/len(att_no))*100)
print("ACC: ",(sum(acc)/len(att_no))*100)

IOU5:  5.333333333333333
IOU10:  5.000000000000001
ACC:  53.333333333333336
